In [1]:
import os
from src.StreamPort.device.analyses import PressureCurvesAnalyses

path = "C:/Users/Sandeep/Desktop/Error-LC/Method-Data"
batches = os.listdir(path)
batches = [os.path.join(path, file) for file in batches]

error_lc_files = []
for batch in batches:
    batch_files = os.listdir(batch)
    batch_files = [os.path.join(batch, file) for file in batch_files if ".D" in file]
    error_lc_files.extend(batch_files)
   
analyses = PressureCurvesAnalyses(files=error_lc_files)
print("Number of analyses: ", len(analyses.data))
%store analyses

Number of analyses:  658
Stored 'analyses' (PressureCurvesAnalyses)


In [ ]:
#%store -r analyses

In [2]:
methods = analyses.get_methods()
print(methods)

['250611_1D_05MLMIN_5-90B_5MIN_SIM-TIC_NOFRAG_POS4.M', '250626_1D_05MLMIN_5-90B_5MIN_SIM-TIC_NOFRAG_IS.M', '250626_1D_05MLMIN_5-90B_5MIN_SIM-TIC_NOFRAG_IS_POS4.M', '250611_1D_05MLMIN_5-90B_5MIN_SIM-TIC_NOFRAG.M']


In [ ]:
methods_plot = analyses.plot_methods()
methods_plot.show()

In [3]:
# error_lc curves belong to the same method
#batch_plot = analyses.plot_batches()
#batch_plot.show()

In [4]:
all_batches = analyses.get_batches()
for i in sorted(all_batches):
    print(i)

250613_Mix-1_10x100ng-mL 2025-06-13 17-02-43
250613_Mix-1_10x10ng-mL 2025-06-13 14-40-12
250616_Mix-1_50x100ng-mL 2025-06-16 10-06-17
250617_Mix-1_50x10ng-mL 2025-06-17 09-19-10
250620_Mix-1_25x100ng-mL 2025-06-20 09-26-34
250623_Mix-1_25x100ng-mL 2025-06-23 15-57-58
250624_Mix-1_20x100ng-mL 2025-06-24 16-17-42
250624_Mix-1_30x100ng-mL 2025-06-24 10-33-02
250625_Mix-1_25x100ng-mL 2025-06-25 07-55-23
250625_Mix-1_35x100ng-mL 2025-06-25 14-38-46
250626_Mix-1_50x100ng-mL 2025-06-26 09-05-34
250702_Mix-1+IS_50x100ng-mL 2025-07-02 09-46-06
250703_Mix-1+IS_10x100ng-mL 2025-07-03 14-14-18
250703_Mix-1+IS_20x100ng-mL 2025-07-03 09-29-09
250704_Mix-1+IS_25x100ng-mL 2025-07-04 10-01-22
250707_Mix-1+IS_20x100ng-mL 2025-07-07 08-54-14
250707_Mix-1+IS_30x100ng-mL 2025-07-07 14-47-33


In [5]:
from datetime import datetime

date_threshold = datetime(2025, 6, 20)
train_batch_names = [batch for batch in all_batches if datetime.strptime(" ".join(batch.split(" ")[-2:]), "%Y-%m-%d %H-%M-%S") < date_threshold]
print("Train batches: ", train_batch_names)

Train batches:  ['250613_Mix-1_10x100ng-mL 2025-06-13 17-02-43', '250616_Mix-1_50x100ng-mL 2025-06-16 10-06-17', '250617_Mix-1_50x10ng-mL 2025-06-17 09-19-10', '250613_Mix-1_10x10ng-mL 2025-06-13 14-40-12']


In [6]:
train_indices = []
for batch in train_batch_names:
    indices = sorted(analyses.get_batch_indices(batch))

    metadata = analyses.get_metadata(indices=indices)
    metadata.sort_values("index", inplace=True)

    metadata = metadata[metadata["batch_position"] > 4]

    indices = sorted(metadata["index"])
    train_indices.extend(indices)

train_indices.remove(4)
train_indices.remove(5)
train_indices.remove(6)
# train_indices.remove(69)
train_indices.remove(124)
train_indices.remove(125)

train_indices.sort()
print("Train indices: ", len(train_indices), " ", train_indices)

Train indices:  135   [7, 8, 9, 10, 11, 12, 13, 14, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]


In [7]:
#fig_train_curves_raw = analyses.plot_pressure_curves(train_indices)
#fig_train_curves_raw.show()

In [8]:
from src.StreamPort.device.methods import PressureCurvesMethodExtractFeaturesNative # relc and resid_noise deactivated for now. Look into weighted features or inverse scaling.

processor = PressureCurvesMethodExtractFeaturesNative() # defaults: (period = 10, window_size = 7, bins = 4, crop = 2)
analyses = processor.run(analyses)

In [9]:
#fig_features=analyses.plot_features(indices = train_indices, normalize=True)

#for trace in fig_features.data:
#    trace.line.color = "black"

#fig_features.show()

In [10]:
train_data = analyses.get_features(train_indices)
train_metadata = analyses.get_metadata(train_indices)

#pre-label known clean data
#train_metadata["error"] = "none"
train_data.to_csv("dev/error_lc_train_features.csv", index=False)
train_metadata.to_csv("dev/error_lc_train_metadata.csv", index=False)

print("Number of training curves: ", len(train_indices))

Number of training curves:  135


In [11]:
train_data.describe()

,runtime,area_0.0_1.75,min_0.0_1.75,max_0.0_1.75,mean_0.0_1.75,std_0.0_1.75,range_0.0_1.75,residual_std_0.0_1.75,roc_0.0_1.75,area_1.75_3.5,...,residual_std_3.5_5.25,roc_3.5_5.25,area_5.251_6.999,min_5.251_6.999,max_5.251_6.999,mean_5.251_6.999,std_5.251_6.999,range_5.251_6.999,residual_std_5.251_6.999,roc_5.251_6.999
count,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,...,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000
mean,420.002141,564.421280,265.928296,331.391704,322.606828,8.001176,65.463407,0.329686,26.016366,530.076302,...,0.088752,20.609288,399.130397,154.734407,319.428852,228.302130,72.981849,164.694444,0.096284,14.462564
std,0.012881,2.782549,1.472798,1.659903,1.590291,0.079290,0.611194,0.180448,7.891431,2.338039,...,0.001257,0.124370,1.635914,0.471150,1.575039,0.935751,0.503807,1.149853,0.001228,0.152080
min,419.990303,559.606170,263.600000,328.500000,319.852618,7.827813,64.295000,0.131237,9.882025,526.253765,...,0.085680,20.326826,396.470007,153.885000,316.620000,226.780362,72.118251,162.420000,0.092900,14.168875
25%,419.990303,561.580779,264.595000,329.745000,320.984432,7.949963,65.027500,0.156466,20.009900,527.729449,...,0.087921,20.507765,397.475537,154.272500,317.847500,227.355533,72.490575,163.655000,0.095429,14.327467
50%,419.990303,564.436750,265.685000,331.370000,322.617672,7.991279,65.390000,0.284336,27.251445,530.127231,...,0.088672,20.609662,399.245146,154.815000,319.510000,228.367712,72.971360,164.710000,0.096250,14.466057
75%,420.015273,566.573360,267.245000,332.780000,323.837251,8.040495,65.785000,0.465169,33.369893,531.881643,...,0.089754,20.690609,400.390867,155.087500,320.655000,229.023128,73.351614,165.585000,0.097175,14.583572
max,420.040242,570.241106,268.950000,334.935000,325.935429,8.319934,68.335000,0.710786,36.330524,535.626908,...,0.091853,20.891547,402.520152,155.710000,322.740000,230.241074,74.058260,167.085000,0.099193,14.820388


In [12]:
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

iso_analyses = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(iso_analyses)


MachineLearningAnalyses 
  variables: 135 rows, 33 columns
  metadata: 135 rows, 14 columns



In [13]:
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn 

scl = MachineLearningScaleFeaturesScalerSklearn(scaler_type= "StandardScaler")

iso_analyses = scl.run(iso_analyses)
#fig_train_features = iso_analyses.plot_data()
#fig_train_features.update_layout(title="Train set features")
#fig_train_features.show()

c:\Users\Sandeep\Desktop\StreamPort\sp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn

iso = MachineLearningMethodIsolationForestSklearn()#force determinism to explain feature importance or for clean model generalization

iso_analyses = iso.run(iso_analyses)
iso_analyses.train()
#fig_train_scores = iso_analyses.plot_scores(threshold="min")
#fig_train_scores.update_layout(title = "Train scores")
#fig_train_scores.show()


In [15]:
#select test set
remaining_batches = list(set(all_batches) - set(train_batch_names))
remaining_batches.sort(key=lambda f: (
    datetime.strptime(f.split(' ')[-2] + '_' + f.split(' ')[-1], "%Y-%m-%d_%H-%M-%S")  # Sort by date
    )  
)
print(remaining_batches)

['250620_Mix-1_25x100ng-mL 2025-06-20 09-26-34', '250623_Mix-1_25x100ng-mL 2025-06-23 15-57-58', '250624_Mix-1_30x100ng-mL 2025-06-24 10-33-02', '250624_Mix-1_20x100ng-mL 2025-06-24 16-17-42', '250625_Mix-1_25x100ng-mL 2025-06-25 07-55-23', '250625_Mix-1_35x100ng-mL 2025-06-25 14-38-46', '250626_Mix-1_50x100ng-mL 2025-06-26 09-05-34', '250702_Mix-1+IS_50x100ng-mL 2025-07-02 09-46-06', '250703_Mix-1+IS_20x100ng-mL 2025-07-03 09-29-09', '250703_Mix-1+IS_10x100ng-mL 2025-07-03 14-14-18', '250704_Mix-1+IS_25x100ng-mL 2025-07-04 10-01-22', '250707_Mix-1+IS_20x100ng-mL 2025-07-07 08-54-14', '250707_Mix-1+IS_30x100ng-mL 2025-07-07 14-47-33']


In [17]:
print(len(remaining_batches))

13


In [ ]:
# new tests sorted by date
tests = {}
for batch in remaining_batches:
    batch_date = datetime.strptime(" ".join(batch.split(" ")[-2:]), "%Y-%m-%d %H-%M-%S")

    if batch_date >= date_threshold: # search and collect test samples
        date_threshold = batch_date 
        
        test_indices = analyses.get_batch_indices(batch)
        if 3 in test_indices:
            #print(analyses.get_metadata(3))
            test_indices.remove(3)
        test_size = len(test_indices)

        for i in range(test_size):
            tests[f"{batch_date.strftime('%Y-%m-%d_%H-%M-%S')}_{i+1}"] = test_indices[i]
            

   index              name                                               path  \
0    657  046-D2F-B2-Blank  C:/Users/Sandeep/Desktop/Error-LC/Method-Data\...   

                                             batch  batch_position  idle_time  \
0  250707_Mix-1+IS_30x100ng-mL 2025-07-07 14-47-33              46       39.0   

  sample    method           timestamp detector    pump          start_time  \
0  Blank  error_lc 2025-07-07 21:06:09       CP  G7167B 2025-07-07 21:06:11   

             end_time     runtime  
0 2025-07-07 21:13:55  420.015273  


In [19]:
dates = list(tests.keys())
print("Test dates: ", dates)

Test dates:  ['2025-06-20_09-26-34_1', '2025-06-20_09-26-34_2', '2025-06-20_09-26-34_3', '2025-06-20_09-26-34_4', '2025-06-20_09-26-34_5', '2025-06-20_09-26-34_6', '2025-06-20_09-26-34_7', '2025-06-20_09-26-34_8', '2025-06-20_09-26-34_9', '2025-06-20_09-26-34_10', '2025-06-20_09-26-34_11', '2025-06-20_09-26-34_12', '2025-06-20_09-26-34_13', '2025-06-20_09-26-34_14', '2025-06-20_09-26-34_15', '2025-06-20_09-26-34_16', '2025-06-20_09-26-34_17', '2025-06-20_09-26-34_18', '2025-06-20_09-26-34_19', '2025-06-20_09-26-34_20', '2025-06-20_09-26-34_21', '2025-06-20_09-26-34_22', '2025-06-20_09-26-34_23', '2025-06-20_09-26-34_24', '2025-06-20_09-26-34_25', '2025-06-20_09-26-34_26', '2025-06-20_09-26-34_27', '2025-06-20_09-26-34_28', '2025-06-20_09-26-34_29', '2025-06-20_09-26-34_30', '2025-06-20_09-26-34_31', '2025-06-20_09-26-34_32', '2025-06-20_09-26-34_33', '2025-06-23_15-57-58_1', '2025-06-23_15-57-58_2', '2025-06-23_15-57-58_3', '2025-06-23_15-57-58_4', '2025-06-23_15-57-58_5', '2025-06-23_

In [20]:
print("Test indices: ", list(tests.values()))

Test indices:  [156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352,

In [21]:
print("Number of test samples: ", len(dates))

Number of test samples:  502


# select test sample by incrementing n below (0 <= n <= 431)

In [22]:
import pandas as pd
import os

In [33]:
for i in range(200, 250):
    n = i
    date = dates[n]
    test_data = analyses.get_features(tests[date])
    test_metadata = analyses.get_metadata(tests[date])

    print(f"Test {date}: index:", tests[date])
    train_indices.append(tests[date])

    #fig_test_features = analyses.plot_features(train_indices, normalize=True)
    #fig_test_features.update_layout(showlegend=False)
    #for trace in fig_test_features.data[:-1]:
    #   trace.line.color = "black"
    #fig_test_features.show()

    #fig_test_features_raw = analyses.plot_features_raw(train_indices)
    #fig_test_features_raw.update_layout(showlegend=False)
    #fig_test_features_raw.show()

    iso_analyses.predict(test_data, test_metadata)

    outliers = iso_analyses.test_prediction_outliers(threshold="min", show_scores=True) # defaults: n_tests = _get_num_tests(), show_scores = False
    #print(outliers["outliers"])

    fig_test_curves = analyses.plot_pressure_curves(train_indices)
    train_indices.remove(tests[date])
    fig_test_curves.update_layout(showlegend=False)
    for trace in fig_test_curves.data[:-1]:
        trace.line.color = "black"
    fig_test_curves.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_curve.png", width=1100, height= 350, scale = 3)
    #fig_test_curves.show()

    fig_test_scores = iso_analyses.plot_scores(threshold="min")
    fig_test_scores.update_layout(title=f"Test set {date} final run")
    fig_test_scores.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_scores.png", width=1100, height= 350, scale = 3)
    #fig_test_scores.show()

    confidence_plot = iso_analyses.plot_confidence_variation()
    #confidence_plot.show()
    confidence_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_confidence_variation.png", width=1100, height= 350, scale = 3)

    threshold_plot = iso_analyses.plot_threshold_variation()
    #threshold_plot.show()
    threshold_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_threshold_variation_serialized_tests.png", width=1100, height= 350, scale = 3)

    train_time_plot = iso_analyses.plot_train_time()
    #train_time_plot.show()
    train_time_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_train_time_variation.png", width=1100, height= 350, scale = 3)

    stability_plot = iso_analyses.plot_model_stability()
    #stability_plot.show()
    stability_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_model_stability_vs_train_size.png", width=1100, height= 350, scale = 3)


    test_data.drop(columns="runtime", inplace=True)
    data = pd.concat([test_metadata, test_data], axis=1)

    if outliers["outliers"]["class"].iloc[0] == "outlier":
        outlier_data = data

        if os.path.exists("dev/error_lc_outliers.csv"):
            outliers_file = pd.read_csv("dev/error_lc_outliers.csv")
            outliers_file = pd.concat([outliers_file, outlier_data], axis = 0)
            outliers_file.drop_duplicates(subset="index", inplace = True, ignore_index=True)
            outliers_file.to_csv("dev/error_lc_outliers.csv", index=False)
        
        else:
            outlier_data.to_csv("dev/error_lc_outliers.csv", index=False)

    else:

        if os.path.exists("dev/error_lc_normals.csv"):
            normals_file = pd.read_csv("dev/error_lc_normals.csv")

            normals_file = pd.concat([normals_file, data], axis = 0)
            normals_file.drop_duplicates(subset="index", inplace=True, ignore_index=True)
            normals_file.to_csv("dev/error_lc_normals.csv", index=False)
        
        else:
            data.to_csv("dev/error_lc_normals.csv", index=False)

    iso_analyses.add_prediction() # add_data already calls self.train()

Test 2025-06-25_14-38-46_36: index: 356
Setting true_classes to majority class...
Test 2025-06-25_14-38-46_37: index: 357
Setting true_classes to majority class...
Test 2025-06-25_14-38-46_38: index: 358
Setting true_classes to majority class...
Test 2025-06-25_14-38-46_39: index: 359
Setting true_classes to majority class...
Test 2025-06-25_14-38-46_40: index: 360
Setting true_classes to majority class...
Test 2025-06-25_14-38-46_41: index: 361
Setting true_classes to majority class...
Test 2025-06-25_14-38-46_42: index: 362
Setting true_classes to majority class...
Test 2025-06-25_14-38-46_43: index: 363
Setting true_classes to majority class...
Test 2025-06-25_14-38-46_44: index: 364
Setting true_classes to majority class...
Test 2025-06-25_14-38-46_45: index: 365
Setting true_classes to majority class...
Test 2025-06-26_09-05-34_1: index: 366
Setting true_classes to majority class...
Test 2025-06-26_09-05-34_2: index: 367
Setting true_classes to majority class...
Test 2025-06-26_09

In [34]:
iso_analyses.get_results(summarize = True)

Index: 156
Train size: 135
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 157
Train size: 135
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 158
Train size: 135
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 159
Train size: 135
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 160
Train size: 135
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 161
Train size: 135
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 162
Train size: 135
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 163
Train size: 135
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 164
Train size: 135
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 165
Train size: 135
Num. Tests: 25
Normal: 25
Outlier: 0

Index: 166
Train size: 136
Num. Tests: 25
Normal: 25
Outlier: 0

Index: 167
Train size: 137
Num. Tests: 25
Normal: 25
Outlier: 0

Index: 168
Train size: 138
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 169
Train size: 138
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 170
Train size: 138
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 171
Train size: 13

In [35]:
plots_list = outliers["score_plots"]
import plotly.graph_objects as go
import random
import itertools

colors = ["blue", "green", "orange", "brown", "black", "purple"]
symbols = ["circle", "square-open", "diamond", "cross", "star", "triangle-up-open"]
sizes = [random.randint(4, 7) for i in range(6)]
cross_product = list(itertools.product(symbols, sizes))

#master_figure = go.Figure()
for i, plot in enumerate(plots_list):
        shape_tuple = random.sample(cross_product, 1)[0]
        color = random.sample(colors, 1)[0]
        for trace in plot["data"]: 
                trace.update(marker=dict(symbol=shape_tuple[0], size=shape_tuple[1], color=color),
                             legendgroup = f"Model{i + 1}",
                             name = f"Model{i + 1}")  
                if trace.mode != "lines":
                        trace.update(line=dict(color = color))
        plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{i+1}_test_scores.png", width=1100, height= 350, scale = 3)

        #master_figure.add_traces(plot["data"])

# master_figure.update_layout(
#         title = "Scores per model",
#         xaxis_title = "Analysis Index",
#         yaxis_title = "U.A / Scores",
#         legend_title = "Models",
#         template = "simple_white"
# )
# master_figure.show()

In [36]:
from PIL import Image
import glob
from datetime import datetime

# 1 sample
parallel_test_gif = []
for i in range(len(plots_list)):
    parallel_test_gif.append(Image.open(f"dev/figures/gif_figs/fig_error_lc_test_{i+1}_test_scores.png"))
parallel_test_gif[0].save("dev/figures/gif_figs/parallel_test_scores.gif", save_all=True, append_images=parallel_test_gif[1:], duration=300, loop=0) # duration = time(ms) for each frame, loop = 0 (infinte loop)


In [37]:

# many samples
curve_pattern = 'dev/figures/gif_figs/fig_error_lc_test_*_curve.png'
curve_gif = glob.glob(curve_pattern)
curve_gif.sort(key=lambda f: (
    datetime.strptime(f.split('_')[5] + '_' + f.split('_')[6], "%Y-%m-%d_%H-%M-%S"),  # Sort by date
    int(f.split('_')[-2])  # Sort by the numeric suffix before '_curve.png'
))
curve_gif = [Image.open(file) for file in curve_gif]
curve_gif[0].save("dev/figures/gif_figs/curves.gif", save_all=True, append_images=curve_gif[1:], duration=300, loop=0)


scores_pattern = 'dev/figures/gif_figs/fig_error_lc_test_*_scores.png'
scores_gif = glob.glob(scores_pattern)
scores_gif = [file for file in scores_gif if "test_scores" not in file]
scores_gif.sort(key=lambda f: (
    datetime.strptime(f.split('_')[5] + '_' + f.split('_')[6], "%Y-%m-%d_%H-%M-%S"),  # Sort by date
    int(f.split('_')[7])  # Sort by the numeric suffix before '_curve.png'
))
scores_gif = [Image.open(file) for file in scores_gif]
scores_gif[0].save("dev/figures/gif_figs/scores.gif", save_all=True, append_images=scores_gif[1:], duration=300, loop=0)


conf_pattern = 'dev/figures/gif_figs/fig_error_lc_test_*_confidence_variation.png'
confidence_gif = glob.glob(conf_pattern)
confidence_gif.sort(key=lambda f: (
    datetime.strptime(f.split('_')[5] + '_' + f.split('_')[6], "%Y-%m-%d_%H-%M-%S"),  # Sort by date
    int(f.split('_')[7])  # Sort by the numeric suffix before '_curve.png'
))
confidence_gif = [Image.open(file) for file in confidence_gif]
confidence_gif[0].save("dev/figures/gif_figs/confidence.gif", save_all=True, append_images=confidence_gif[1:], duration=300, loop=0)


thresh_pattern = 'dev/figures/gif_figs/fig_error_lc_test_*_threshold_variation_serialized_tests.png'
threshold_gif = glob.glob(thresh_pattern)
threshold_gif.sort(key=lambda f: (
    datetime.strptime(f.split('_')[5] + '_' + f.split('_')[6], "%Y-%m-%d_%H-%M-%S"),  # Sort by date
    int(f.split('_')[7])  # Sort by the numeric suffix before '_curve.png'
))
threshold_gif = [Image.open(file) for file in threshold_gif]
threshold_gif[0].save("dev/figures/gif_figs/thresholds.gif", save_all=True, append_images=threshold_gif[1:], duration=300, loop=0)


time_pattern = 'dev/figures/gif_figs/fig_error_lc_test_*_train_time_variation.png'
train_time_gif = glob.glob(time_pattern)
train_time_gif.sort(key=lambda f: (
    datetime.strptime(f.split('_')[5] + '_' + f.split('_')[6], "%Y-%m-%d_%H-%M-%S"),  # Sort by date
    int(f.split('_')[7])  # Sort by the numeric suffix before '_curve.png'
))
train_time_gif = [Image.open(file) for file in train_time_gif]
train_time_gif[0].save("dev/figures/gif_figs/train_times.gif", save_all=True, append_images=train_time_gif[1:], duration=300, loop=0)


stability_pattern = 'dev/figures/gif_figs/fig_error_lc_test_*_model_stability_vs_train_size.png'
stability_gif = glob.glob(stability_pattern)
stability_gif.sort(key=lambda f: (
    datetime.strptime(f.split('_')[5] + '_' + f.split('_')[6], "%Y-%m-%d_%H-%M-%S"),  # Sort by date
    int(f.split('_')[7])  # Sort by the numeric suffix before '_curve.png'
))
stability_gif = [Image.open(file) for file in stability_gif]
stability_gif[0].save("dev/figures/gif_figs/stabilities.gif", save_all=True, append_images=stability_gif[1:], duration=300, loop=0)

# KNN

In [ ]:

# batch_list = pd.read_excel("C:/Users/Sandeep/Desktop/Error-LC/Batch-Liste.xlsx")
# error_batches = batch_list[batch_list["provoked_error"].notna() | batch_list["error_timestamp"].notna()]

# error_batches[["batch_name", "start_date"]] = error_batches[["batch_name", "start_date"]].ffill()
# error_batches.head()

In [ ]:
# # create random splits of training and test sets. Shuffling and train-test-splitting with 'stratify' ensure a good distribution of classes so KNN learning is effective
# from sklearn.model_selection import train_test_split
# features_train, features_test, metadata_train, metadata_test = train_test_split(labeled_data, metadata, test_size=0.3, stratify=metadata["label"])

# #%store features_train features_test metadata_train metadata_test

In [ ]:

# from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses
# knn_ana = MachineLearningAnalyses(variables=features_train, metadata=metadata_train)

# from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn
# scl = MachineLearningScaleFeaturesScalerSklearn(scaler_type="StandardScaler")
# knn_ana = scl.run(knn_ana)

# from src.StreamPort.machine_learning.methods import MachineLearningMethodNearestNeighboursClassifierSklearn
# knn = MachineLearningMethodNearestNeighboursClassifierSklearn(n_neighbors=1) #odd number for binary classification. 1 usually causes overfit, could be suitable for small amount of data
# knn_ana = knn.run(knn_ana)

# print(knn_ana)

In [ ]:
# knn_ana.plot_data()

In [ ]:
# knn_ana.train()
# train_classes = knn_ana.get_training_labels()
# print(train_classes)

In [ ]:
# train_labels = knn_ana.get_true_labels("train")
# print("True labels: ",train_labels.shape)#, " ", train_labels)
# print("Metadata labels: ", metadata_train[["index","label"]].shape)#, " ", metadata_train[["index","label"]])
# if not train_labels.equals(train_classes):
#     print("mismatch")
# else:
#     print("match")

In [ ]:
# import umap
# import numpy as np
# import sklearn.preprocessing as scaler
# import plotly.graph_objects as go

# scaler = scaler.StandardScaler()
# features_train_scaled = scaler.fit_transform(features_train)

# # UMAP for dimensionality reduction
# umap_train_model = umap.UMAP(n_components=2)
# features_test_reduced_umap = umap_train_model.fit_transform(features_train_scaled)

# # Prepare the reduced data frame
# reduced_data_umap = pd.DataFrame(features_test_reduced_umap, columns=['UMAP-1', 'UMAP-2'])
# reduced_data_umap["predicted_labels"] = train_classes["label"]
# reduced_data_umap["true_labels"] = train_labels["label"]
# reduced_data_umap["index"] = train_labels["index"]

# # Convert labels to binary values (0 for normal, 1 for outlier)
# #reduced_data_umap["predicted_labels"] = np.where(reduced_data_umap["predicted_labels"] == "normal", 0, 1)
# #reduced_data_umap["true_labels"] = np.where(reduced_data_umap["true_labels"] == "normal", 0, 1)

# # Initialize the UMAP plot
# fig_umap = go.Figure()

# # Plot true labels (blue circles)
# for i in range(len(reduced_data_umap)):
#     if reduced_data_umap["true_labels"][i] == "outlier":
#         fig_umap.add_trace(
#             go.Scatter(
#                 x=[reduced_data_umap["UMAP-1"][i]],
#                 y=[reduced_data_umap["UMAP-2"][i]],
#                 mode="markers",
#                 marker=dict(
#                     color="blue",
#                     size=10,
#                     opacity=0.7,
#                     symbol="circle",
#                 ),
#                 name="True labels",
#                 hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
#             )
#         )

#     elif reduced_data_umap["true_labels"][i] == "normal":
#         fig_umap.add_trace(
#             go.Scatter(
#                 x=[reduced_data_umap["UMAP-1"][i]],
#                 y=[reduced_data_umap["UMAP-2"][i]],
#                 mode="markers",
#                 marker=dict(
#                     color="white",  # Color for normal labels
#                     size=10,
#                     opacity=0.4,
#                     symbol="circle",
#                 ),
#                 hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
#             )
#         )

# # Plot predicted labels (red 'x's)
# for i in range(len(reduced_data_umap)):
#     if reduced_data_umap["predicted_labels"][i] == "outlier":
#         fig_umap.add_trace(
#             go.Scatter(
#                 x=[reduced_data_umap["UMAP-1"][i]],
#                 y=[reduced_data_umap["UMAP-2"][i]],
#                 mode="markers",
#                 marker=dict(
#                     color="red",
#                     size=8,
#                     opacity=0.5,
#                     symbol="x",
#                 ),
#                 name="Predicted Labels",
#                 hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
#             )
#         )

#     elif reduced_data_umap["predicted_labels"][i] == "normal":
#         fig_umap.add_trace(
#             go.Scatter(
#                 x=[reduced_data_umap["UMAP-1"][i]],
#                 y=[reduced_data_umap["UMAP-2"][i]],
#                 mode="markers",
#                 marker=dict(
#                     color="white",  # Color for normal predicted labels
#                     size=8,
#                     opacity=0.4,
#                     symbol="x",
#                 ),
#                 hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
#             )
#         )

# # Update layout for the UMAP plot
# fig_umap.update_layout(
#     title="KNN Train Data Classification using UMAP reduction",
#     xaxis_title="UMAP-1",
#     yaxis_title="UMAP-2",
#     template="plotly_dark",
#     showlegend=False,
#     hovermode="closest"
# )

# fig_umap.show()

In [ ]:
# knn_ana.predict(features_test, metadata_test)
# classes = knn_ana.get_prediction_labels()
# classes.reset_index(drop=True, inplace=True)
# print("Predicted classes: ", classes.shape)#, " ", classes)

In [ ]:
# true_labels = knn_ana.get_true_labels()
# true_labels.reset_index(drop=True, inplace=True)
# print("True classes: ", true_labels.shape)#, " ", true_labels)

In [ ]:
# # probabilities will sub for kNN where confidence was used for iForest 
# probs = knn_ana.get_prediction_probabilities()
# # print("Prediction probabilities: ", probs) # n_neighbours = 1 is not ideal but provides best results for small datasets. Use GridSearchCV (see dev_model_comparison) to find best params

In [ ]:
# # UMAP for dimensionality reduction while maintaining local relationships
# umap_test_model = umap.UMAP(n_components=2)
# features_test_scaled = scaler.fit_transform(features_test)
# features_test_reduced_umap = umap_test_model.fit_transform(features_test_scaled)

# # Prepare the reduced data frame
# reduced_data_umap = pd.DataFrame(features_test_reduced_umap, columns=['UMAP-1', 'UMAP-2'])
# reduced_data_umap["predicted_labels"] = classes["label"]
# reduced_data_umap["true_labels"] = true_labels["label"]
# reduced_data_umap["index"] = true_labels["index"]

# # Convert labels to binary values (0 for normal, 1 for outlier)
# #reduced_data_umap["predicted_labels"] = np.where(reduced_data_umap["predicted_labels"] == "normal", 0, 1)
# #reduced_data_umap["true_labels"] = np.where(reduced_data_umap["true_labels"] == "normal", 0, 1)

# # Initialize the UMAP plot
# fig_umap = go.Figure()

# # Plot true labels (blue circles)
# for i in range(len(reduced_data_umap)):
#     if reduced_data_umap["true_labels"][i] == "outlier":
#         fig_umap.add_trace(
#             go.Scatter(
#                 x=[reduced_data_umap["UMAP-1"][i]],
#                 y=[reduced_data_umap["UMAP-2"][i]],
#                 mode="markers",
#                 marker=dict(
#                     color="blue",
#                     size=10,
#                     opacity=0.7,
#                     symbol="circle",
#                 ),
#                 name="True labels",
#                 hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
#             )
#         )

#     elif reduced_data_umap["true_labels"][i] == "normal":
#         fig_umap.add_trace(
#             go.Scatter(
#                 x=[reduced_data_umap["UMAP-1"][i]],
#                 y=[reduced_data_umap["UMAP-2"][i]],
#                 mode="markers",
#                 marker=dict(
#                     color="white",  # Color for normal labels
#                     size=10,
#                     opacity=0.4,
#                     symbol="circle",
#                 ),
#                 hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
#             )
#         )

# # Plot predicted labels (red 'x's)
# for i in range(len(reduced_data_umap)):
#     if reduced_data_umap["predicted_labels"][i] == "outlier":
#         fig_umap.add_trace(
#             go.Scatter(
#                 x=[reduced_data_umap["UMAP-1"][i]],
#                 y=[reduced_data_umap["UMAP-2"][i]],
#                 mode="markers",
#                 marker=dict(
#                     color="red",
#                     size=8,
#                     opacity=0.5,
#                     symbol="x",
#                 ),
#                 name="Predicted Labels",
#                 hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
#             )
#         )

#     elif reduced_data_umap["predicted_labels"][i] == "normal":
#         fig_umap.add_trace(
#             go.Scatter(
#                 x=[reduced_data_umap["UMAP-1"][i]],
#                 y=[reduced_data_umap["UMAP-2"][i]],
#                 mode="markers",
#                 marker=dict(
#                     color="white",  # Color for normal predicted labels
#                     size=8,
#                     opacity=0.4,
#                     symbol="x",
#                 ),
#                 hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
#             )
#         )

# # Update layout for the UMAP plot
# fig_umap.update_layout(
#     title="KNN Test Classification Results using UMAP reduction",
#     xaxis_title="UMAP-1",
#     yaxis_title="UMAP-2",
#     template="plotly_dark",
#     showlegend=False,
#     hovermode="closest"
# )

# fig_umap.show()
# #fig_umap.write_image("dev/figures/fig_knn_test_predictions_umap.png", width=1100, height= 350, scale = 3)